In [1]:
##!pip3 install torch

In [2]:
##!pip3 install numpy

In [3]:
##!pip3 install nltk

In [4]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam

import numpy as np
import scipy.sparse as sp
from topic_modeling import VAE, Encoder, Decoder, SparseDataset, topic_cos_sim



In [5]:
ensure_pyterrier_is_loaded()
tira = Client()

# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [6]:
# hyperparameters

cuda = False
device = torch.device("cuda" if cuda else "cpu")

epochs = 20
#epochs = 4
batch_size = 1024
#batch_size = 8
learning_rate = 1e-2
#learning_rate = 1e-3

In [7]:
""" # bow
# bow = {'O02-2002' : {'i': 2, 'mixtur': 1, 'adopt': 1, 'exampl': 1, 'featur': 2, 'especi': 1, 'classif': 1, 'requir': 1, 'gener': 1, 'measur': 2, 'idf': 1, 'characterist': 1, 'valu': 2, 'languag': 1, 'vector': 3, 'environ': 1, 'data': 1, 'turn': 1}}


bow = {}
for docid in range(doi.getNumberOfDocuments()):
  for posting in  di.getPostings(doi.getDocumentEntry(docid)):
    docno = meta.getItem("docno", docid)
    if not docno in bow:
      bow[docno]= {}
    termid = posting.getId()
    lee = lex.getLexiconEntry(termid)
    bow[docno][lee.getKey()] = posting.getFrequency()

print(bow["O02-2002"]) """

' # bow\n# bow = {\'O02-2002\' : {\'i\': 2, \'mixtur\': 1, \'adopt\': 1, \'exampl\': 1, \'featur\': 2, \'especi\': 1, \'classif\': 1, \'requir\': 1, \'gener\': 1, \'measur\': 2, \'idf\': 1, \'characterist\': 1, \'valu\': 2, \'languag\': 1, \'vector\': 3, \'environ\': 1, \'data\': 1, \'turn\': 1}}\n\n\nbow = {}\nfor docid in range(doi.getNumberOfDocuments()):\n  for posting in  di.getPostings(doi.getDocumentEntry(docid)):\n    docno = meta.getItem("docno", docid)\n    if not docno in bow:\n      bow[docno]= {}\n    termid = posting.getId()\n    lee = lex.getLexiconEntry(termid)\n    bow[docno][lee.getKey()] = posting.getFrequency()\n\nprint(bow["O02-2002"]) '

In [8]:
di = index.getDirectIndex()
doi = index.getDocumentIndex()
lex = index.getLexicon()
meta = index.getMetaIndex()

test = True

if test:
  train_size = 64
  val_size = 32
else:
  train_size = int(doi.getNumberOfDocuments() * 0.8)
  val_size = doi.getNumberOfDocuments() - int(doi.getNumberOfDocuments() * 0.8)

bow_sparse = sp.dok_matrix((train_size,len(lex)), dtype=np.int8)
bow_sparse_val = sp.dok_matrix((val_size,len(lex)), dtype=np.int8)

for docid in range(train_size + val_size):
  docno = meta.getItem("docno", docid)
  # if docid == 0:
    # print(docno)
  for posting in  di.getPostings(doi.getDocumentEntry(docid)):
    if posting.getFrequency() > 0:
      termid = posting.getId()
      lee = lex.getLexiconEntry(termid)
      # if docid == 0:
        # print(f"{lee.getKey()} , {termid}")
      if docid < train_size:
        bow_sparse[docid, termid] = posting.getFrequency()
      else:
        bow_sparse_val[docid - train_size, termid] = posting.getFrequency()
      # if docid == 0:
        # print(f"{docid}")
        # print(f"{bow_sparse[docid, termid]}")

bow_sparse = bow_sparse.tocsr()
bow_sparse_val = bow_sparse_val.tocsr()

In [9]:
#print(bow_sparse[31])
print(bow_sparse.shape)
print(bow_sparse[31].todense().item(117))

print(bow_sparse_val.shape)
print(bow_sparse_val[31].todense().item(557))

(64, 97223)
7
(32, 97223)
4


In [10]:
"""dim = bow_sparse.shape
device = torch.device(device)
indptr = torch.tensor(bow_sparse.indptr, dtype=torch.int64, device=device)
indices = torch.tensor(bow_sparse.indices, dtype=torch.int64, device=device)
data = torch.tensor(bow_sparse.data, dtype=torch.int8, device=device)

print(dim[0])

idx = 0

obs = torch.zeros((dim[1],), dtype=torch.int8, device=device)
print(obs)
print(len(obs))
print(obs[0])
ind1,ind2 = indptr[idx],indptr[idx+1]
obs[indices[ind1:ind2]] = data[ind1:ind2]
print(obs.shape)
print(obs)"""

'dim = bow_sparse.shape\ndevice = torch.device(device)\nindptr = torch.tensor(bow_sparse.indptr, dtype=torch.int64, device=device)\nindices = torch.tensor(bow_sparse.indices, dtype=torch.int64, device=device)\ndata = torch.tensor(bow_sparse.data, dtype=torch.int8, device=device)\n\nprint(dim[0])\n\nidx = 0\n\nobs = torch.zeros((dim[1],), dtype=torch.int8, device=device)\nprint(obs)\nprint(len(obs))\nprint(obs[0])\nind1,ind2 = indptr[idx],indptr[idx+1]\nobs[indices[ind1:ind2]] = data[ind1:ind2]\nprint(obs.shape)\nprint(obs)'

In [11]:
train_dataset = SparseDataset(bow_sparse, device)
val_dataset = SparseDataset(bow_sparse_val, device)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [12]:
print(train_dataset.__getitem__(0))

tensor([2., 1., 1.,  ..., 0., 0., 0.])


In [13]:
""" for batch, x in enumerate(train_loader):
    print(batch)
    for doc in x:
        for freq in doc:
            if freq > 0:
                print(freq)
                break;
    break """

' for batch, x in enumerate(train_loader):\n    print(batch)\n    for doc in x:\n        for freq in doc:\n            if freq > 0:\n                print(freq)\n                break;\n    break '

In [14]:
# more hyperparameters

input_dim  = len(lex)
#hidden_dim = 512
#latent_dim = 8

hidden_dim = 16384
latent_dim = 128


In [15]:
encoder = Encoder(input_dim=input_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = input_dim)

vae = VAE(encoder=encoder, decoder=decoder, device=device).to(device)

In [16]:
print(vae)

VAE(
  (encoder): Encoder(
    (lin1): Linear(in_features=97223, out_features=512, bias=True)
    (lin2): Linear(in_features=512, out_features=512, bias=True)
    (mean): Linear(in_features=512, out_features=8, bias=True)
    (var): Linear(in_features=512, out_features=8, bias=True)
    (LeakyReLU): LeakyReLU(negative_slope=0.2)
  )
  (decoder): Decoder(
    (lin1): Linear(in_features=8, out_features=512, bias=True)
    (lin2): Linear(in_features=512, out_features=512, bias=True)
    (out): Linear(in_features=512, out_features=97223, bias=True)
    (LeakyReLU): LeakyReLU(negative_slope=0.2)
  )
)


In [17]:
def loss_function(x, x_hat, mean, log_var):
    logsoftmax = torch.log_softmax(x_hat,dim=1)
    rec_loss = -1.0 * torch.sum(x*logsoftmax)

    kl_div = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

    return rec_loss + kl_div

optimizer = Adam(vae.parameters(), lr=learning_rate)

In [18]:
print("VAE training:")
vae.train()

for epoch in range(epochs):
    overall_loss = 0
    for batch, x in enumerate(train_loader):

        x = x.view(batch_size, input_dim)
        x = x.to(device)

        optimizer.zero_grad()

        x_hat, mean, log_var = vae(x)
        
        loss = loss_function(x, x_hat, mean, log_var)
        
        overall_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch + 1} :\tAverage Loss: {overall_loss / (batch * batch_size)}")
    
print("VAE training done")

VAE training:
Epoch 1 :	Average Loss: 772.3530011858259
Epoch 2 :	Average Loss: 772.3189958844866
Epoch 3 :	Average Loss: 771.6951032366071
Epoch 4 :	Average Loss: 771.0533447265625
VAE training done


In [19]:
vae.eval()

with torch.no_grad():
    for batch, x in enumerate(val_loader):
        x = x.view(batch_size, input_dim)
        x = x.to(device)
        
        x_hat, mean, log_var = vae(x)
        for doc in x_hat:
            print(len(doc))
            print(doc)
            print(vae.inference_theta(doc))
            break

97223
tensor([1.0252e-09, 3.3232e-10, 1.7760e-09,  ..., 1.9196e-11, 2.4006e-11,
        1.0348e-11])
[0.0780568  0.14716716 0.05147269 0.00470103 0.16613457 0.1580446
 0.29031318 0.10410991]
97223
tensor([1.4639e-07, 4.8486e-08, 1.7709e-07,  ..., 6.1475e-09, 6.5611e-09,
        4.4567e-09])
[0.5876864  0.02900825 0.0537922  0.05605497 0.00761662 0.16820033
 0.06223368 0.03540755]
97223
tensor([1.1710e-08, 3.5746e-09, 2.2613e-08,  ..., 3.1372e-10, 3.8767e-10,
        1.5317e-10])
[0.10242098 0.13871805 0.03407496 0.04700743 0.20096597 0.17763911
 0.0315076  0.267666  ]
97223
tensor([9.1135e-11, 3.1194e-11, 1.4772e-10,  ..., 1.0660e-12, 1.5897e-12,
        7.0705e-13])
[0.09854765 0.0228593  0.01067117 0.7239667  0.01772923 0.03172873
 0.04491218 0.04958499]


In [20]:

qe = (pt.BatchRetrieve(index, wmodel="BM25") % 10 >> \
      pt.apply.doc_score(lambda doc : topic_cos_sim(doc, index, vae)))

run = qe(pt_dataset.get_topics('text'))

run.head(10)

AttributeError: module 'numpy' has no attribute 'norm'